In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, MeanShift, estimate_bandwidth
from sklearn.datasets import make_classification

import matplotlib as mpl

In [3]:
df = pd.read_csv('household_power_consumption.txt',sep = ';',
                parse_dates={'dt':['Date','Time']},
                infer_datetime_format=True,
                low_memory=False, na_values=['nan','?'],
                index_col='dt')

FileNotFoundError: [Errno 2] No such file or directory: 'household_power_consumption.txt'

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df["Global_active_power"]=df["Global_active_power"].fillna(df["Global_active_power"].mode()[0])
df["Global_reactive_power"]=df["Global_reactive_power"].fillna(df["Global_reactive_power"].mode()[0])
df["Voltage"]=df["Voltage"].fillna(df["Voltage"].mode()[0])
df["Global_intensity"]=df["Global_intensity"].fillna(df["Global_intensity"].mode()[0])
df["Sub_metering_1"]=df["Sub_metering_1"].fillna(df["Sub_metering_1"].mode()[0])
df["Sub_metering_2"]=df["Sub_metering_2"].fillna(df["Sub_metering_2"].mode()[0])
df["Sub_metering_3"]=df["Sub_metering_3"].fillna(df["Sub_metering_3"].mode()[0])


In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
from sklearn.metrics import silhouette_score
silhouette_scores = [] 

for n_cluster in range(2,75):
    silhouette_scores.append( 
        silhouette_score(df, KMeans(n_clusters = n_cluster).fit_predict(df))) 
    
# Plotting a bar graph to compare the results 
k = [2, 3, 4, 5, 6,7,8,9,10,11,12,13,14] 
plt.bar(k, silhouette_scores) 
plt.xlabel('Number of clusters', fontsize = 10) 
plt.ylabel('Silhouette Score', fontsize = 10) 
plt.grid()
plt.show()

In [ ]:
df.isnull().sum()

In [ ]:


sns.pairplot(df)

In [ ]:
from sklearn.cluster import KMeans
model = KMeans(n_clusters=4)

In [ ]:
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, MeanShift, estimate_bandwidth
from sklearn.datasets import make_classification

import matplotlib.pyplot as plt
import seaborn as sns
from numpy import unique
from numpy import where

In [ ]:
columns = list(df.columns)

scaler = MinMaxScaler(feature_range=(0,1))
df=df.copy()
df = scaler.fit_transform(df)
df = pd.DataFrame(df, columns=columns)
df_sample = df[:5000]

In [ ]:
kmeans = KMeans(n_clusters=5)
df=df.copy()
k_fit = kmeans.fit(df)

pd.options.display.max_columns = 10
predictions = k_fit.labels_
df['Clusters'] = predictions
df.head()

In [ ]:
df.Clusters.value_counts()

In [ ]:
pca = PCA(n_components=2)
pca_fit = pca.fit_transform(df)

df_pca = pd.DataFrame(data=pca_fit, columns=['PCA1', 'PCA2'])
df_pca['Clusters'] = predictions
df_pca.head()

In [ ]:
sillhoute_scores = []

X = df.values.copy()
    
scaler = MinMaxScaler(feature_range=(0,1))
X = scaler.fit_transform(X)

In [ ]:
kmeans = KMeans(n_clusters=10)
cluster_found = kmeans.fit_predict(X)
cluster_found_sr = pd.Series(cluster_found, name='cluster')
df_uci_pivot = df.set_index(cluster_found_sr, append=True )

fig, ax= plt.subplots(1,1, figsize=(18,10))
color_list = ['blue','red','green','yellow','purple','orange']
cluster_values = sorted(df.index.get_level_values(1).unique())

for cluster, color in zip(cluster_values, color_list):
    df_uci_pivot.xs(cluster, level=1).T.plot(
        ax=ax, legend=False, alpha=0.01, color=color, label= f'Cluster {cluster}'
        )
    df_uci_pivot.xs(cluster, level=1).median().plot(
        ax=ax, color=color, alpha=0.9, ls='--'
    )

ax.set_xticks(np.arange(1,25))
ax.set_ylabel('kilowatts')
ax.set_xlabel('hour')
ax.legend()